In [1]:
%%capture
!pip install unidecode
!pip install nltk
!pip install tensorflow
!pip install sklearn
!pip install gensim

In [2]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import re
import unidecode
import warnings
import seaborn as sns
import time
import pickle
import gensim
import nltk
import os
import random
nltk.download('stopwords')

from nltk.corpus import stopwords
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

sw = set(stopwords.words('english'))
os.environ['PYTHONHASHSEED']=str(2109)
tf.random.set_seed(2109)
random.seed(2109)
warnings.filterwarnings('ignore')
np.random.seed(2109)
sns.set_style('dark')

2021-10-15 16:30:24.193855: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-15 16:30:24.193891: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package stopwords to /home/greca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

len(model.wv.vocab)

embeddings_index = {}

f = open('glove.twitter.27B.200d.txt')
for line in f:
    values = line.split()
    word = value = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' %len(embeddings_index))

In [4]:
data = pd.read_csv('train.csv')

In [5]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(data['tweet'])
data['tokenized'] = tokenizer.texts_to_sequences(data['tweet'])

vocab_size = len(tokenizer.word_index) + 1
vocab_size

45890

In [6]:
EMBEDDING_DIM = 300

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except:
        embedding_matrix[i] = np.random.randn(EMBEDDING_DIM)

In [7]:
len(embedding_matrix)

45890

In [8]:
with open(r"word2vec.pickle", "wb") as output_file:
    pickle.dump(embedding_matrix, output_file, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
def remove_username(text):
  text = re.sub(r'\@[^\s]+', ' ', text)
  return text

def remove_newline(text):
  text = text.replace('\n', ' ')
  return text

def only_letters(text):
  text = re.sub(r'[^a-záâàãéêèẽíìîĩóòõôúùũû\s]+', ' ', text)
  return text

def remove_link(text):
  text = re.sub(r'www\.?[^\s]+', ' ', text)
  return text

def remove_hyperlink(text):
  text = re.sub(r'\<.?\>', ' ', text)
  return text

def remove_accent(text):
  text = unidecode.unidecode(text)
  return text

def adjustment_text(text):
  text = re.sub(r'\s+', ' ', text)
  text = text.strip()
  return text

def remove_stopwords(text):
  text = [word for word in text.split() if word not in sw]
  text = ' '.join(text)
  return text

def remove_spam(text):
  text = re.sub(r'\&amp', ' ', text)
  text = re.sub(r'\&lt', ' ', text)
  text = re.sub(r'\&gt', ' ', text)
  text = re.sub(r'\#follow|\#followme|\#like|\#f4f|\#photooftheday', ' ', text)
  return text

def remove_slangs(text):
  text = re.sub(r' b4 ', ' before ', text)
  text = re.sub(r' 2b ', ' to be ', text)
  text = re.sub(r' 2morrow ', ' tomorrow ', text)
  text = re.sub(r' rn ', ' right now ', text)
  text = re.sub(r' brb ', ' be right back ', text)
  text = re.sub(r' mb ', ' my bad ', text)
  text = re.sub(r' luv ', ' love ', text)
  text = re.sub(r' b ', ' be ', text)
  text = re.sub(r' r ', ' are ', text)
  text = re.sub(r' u ', ' you ', text)
  text = re.sub(r' y ', ' why ', text)
  text = re.sub(r' ur ', ' your ', text)
  text = re.sub(r' hbd ', ' happy birthday ', text)
  text = re.sub(r' bday ', ' birthday ', text)
  text = re.sub(r' bihday ', ' birthday ', text)
  text = re.sub(r' omg ', ' oh my god ', text)
  text = re.sub(r' lol ', ' laughing out loud ', text)
  return text

def remove_abbreviations(text):
  text = re.sub(r" can\'t ", " can not ", text)
  text = re.sub(r" i\'m ", " i am ", text)
  text = re.sub(r" i\'ll ", " i will ", text)
  text = re.sub(r" i\'d ", " i would ", text)
  text = re.sub(r" i\'ve ", " i have ", text)
  text = re.sub(r" ain\'t ", " am not ", text)
  text = re.sub(r" haven\'t ", " have not ", text)
  text = re.sub(r" hasn\'t ", " has not ", text)
  text = re.sub(r" can\'t ", " can not ", text)
  text = re.sub(r" won\'t ", " will not ", text)
  text = re.sub(r" you\'re ", " you are ", text)
  text = re.sub(r" we\'re ", " we are ", text)
  text = re.sub(r" they\'re ", " they are ", text)
  text = re.sub(r" he\'s ", " he is ", text)
  text = re.sub(r" she\'s ", " she is ", text)
  text = re.sub(r" it\'s ", " it is ", text)
  text = re.sub(r" don\'t ", " do not ", text)
  text = re.sub(r" doesn\'t ", " does not ", text)
  text = re.sub(r" wouldn\'t ", " would not ", text)
  text = re.sub(r" couldn\'t ", " could not ", text)
  text = re.sub(r" shouldn\'t ", " should not ", text)
  return text

def remove_one_len_word(text):
  text = re.sub(r'\b[a-z]\b', ' ', text)
  return text

def preprocessing(data):
  data['cleaned_tweet'] = data['tweet'].apply(str)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(lambda x: x.lower())
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_newline)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_hyperlink)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_spam)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_link)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_username)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_abbreviations)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(only_letters)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_accent)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_slangs)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_stopwords)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_one_len_word)
  data['cleaned_tweet'] = data['cleaned_tweet'].apply(adjustment_text)
  return data

In [10]:
old_data = data.copy()

In [11]:
data = pd.read_csv('train.csv')

In [12]:
data = preprocessing(data)
data = data.drop_duplicates()
data = data.dropna()
data = data.drop(columns=['tweet'])
data = data.rename(columns={'cleaned_tweet': 'tweet'})
data.head(15)

,id,label,tweet
0,1,0,father dysfunctional selfish drags kids dysfun...
1,2,0,thanks lyft credit use cause offer wheelchair ...
2,3,0,birthday majesty
3,4,0,model love take time
4,5,0,factsguide society motivation
5,6,0,huge fan fare big talking leave chaos pay disp...
6,7,0,camping tomorrow dannya
7,8,0,next school year year exams think school exams...
8,9,0,love land allin cavs champions cleveland cleve...
9,10,0,welcome gr


In [13]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(data['tweet'])
data['tokenized'] = tokenizer.texts_to_sequences(data['tweet'])

vocab_size = len(tokenizer.word_index) + 1

In [14]:
print(vocab_size)

38843


In [15]:
EMBEDDING_DIM = 300

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except:
        embedding_matrix[i] = np.random.randn(EMBEDDING_DIM)

In [16]:
len(embedding_matrix)

38843

In [17]:
with open(r"word2vec_preprocessing.pickle", "wb") as output_file:
    pickle.dump(embedding_matrix, output_file, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
all_data = pd.concat([old_data, data], axis=0)
all_data = all_data.reset_index(drop=True)
all_data.head()

,id,label,tweet,tokenized
0,1,0,@user when a father is dysfunctional and is s...,"[1, 34, 4, 252, 11, 16533, 7, 11, 20, 2753, 97..."
1,2,0,@user @user thanks for #lyft credit i can't us...,"[1, 1, 170, 9, 5701, 2600, 5, 68, 452, 680, 69..."
2,3,0,bihday your majesty,"[62, 26, 3422]"
3,4,0,#model i love u take with u all the time in ...,"[140, 5, 13, 37, 76, 16, 37, 21, 2, 39, 8, 162..."
4,5,0,factsguide: society now #motivation,"[2946, 1621, 49, 299]"


In [19]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(all_data['tweet'].values)
all_data['tokenized'] = tokenizer.texts_to_sequences(all_data['tweet'].values)

vocab_size = len(tokenizer.word_index) + 1
vocab_size

49175

In [20]:
print(vocab_size)

49175


In [21]:
EMBEDDING_DIM = 300

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except:
        embedding_matrix[i] = np.random.randn(EMBEDDING_DIM)

In [22]:
len(embedding_matrix)

49175

In [23]:
with open(r"word2vec_total.pickle", "wb") as output_file:
    pickle.dump(embedding_matrix, output_file, protocol=pickle.HIGHEST_PROTOCOL)